In [1]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors
import seaborn as sns
import matplotlib.pyplot as plt

In [15]:
blind = pd.read_csv("../../Data/Data_submission/test_blind.csv", sep=";") #fichier csv qui sera merge pas les prédiction et soumis sur Kaggle

In [16]:
blind

,Id,datasetName,date,dayOfYear,year,patchID,timeSerieID,lon,lat,x_EPSG3035,y_EPSG3035,geoUncertaintyInM
0,1,CBNMed,1629064800,228,2021,10125,3988380,1.729607,42.52171,3.640762e+06,2.194746e+06,5.0
1,2,CBNMed,1629064800,228,2021,10126,3997090,1.729624,42.52177,3.640764e+06,2.194753e+06,4.0
2,3,CBNMed,1629064800,228,2021,10127,4001106,1.729652,42.52197,3.640769e+06,2.194775e+06,5.0
3,4,CBNMed,1629064800,228,2021,10129,4004605,1.729671,42.52192,3.640770e+06,2.194769e+06,6.0
4,5,CBNMed,1531346400,193,2018,10154,3987660,1.826847,42.57452,3.649316e+06,2.199695e+06,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
22399,22400,Inventaire forestier IGN,2018-03-14,73,2018,3018864,3990152,9.421020,42.38755,4.273164e+06,2.142680e+06,1.0
22400,22401,Inventaire forestier IGN,2018-03-22,81,2018,3018865,3986840,9.429860,42.26077,4.273796e+06,2.128639e+06,1.0
22401,22402,Inventaire forestier IGN,2017-03-13,72,2017,3018866,3993669,9.462680,42.92578,4.277004e+06,2.202255e+06,1.0
22402,22403,Inventaire forestier IGN,2017-03-15,74,2017,3018867,3993567,9.497790,42.41844,4.279528e+06,2.146054e+06,1.0


In [6]:
blind = pd.read_csv("../../Data/Data_submission/test_blind.csv", sep=";") #fichier csv qui sera merge pas les prédiction et soumis sur Kaggle
donnees_abiotiques = pd.read_csv("../../Data/Data_train/enviroTab_pa_train.csv", sep = ';')  #fichier csv données abiotiques des patchID
blind_abiotique = pd.read_csv("../../Data/Data_submission/enviroTab_pa_test.csv", sep = ';')
presences = pd.read_excel("../../Data/Data_train/Presences_Absences_train.xlsx") #fichier excel presences/absences

In [9]:
X1 = blind_abiotique[['dayOfYear', 'bio1', 'bio2',
       'bio3', 'bio4', 'bio5', 'bio6', 'bio7', 'bio8', 'bio9', 'bio10',
       'bio11', 'bio12', 'bio13', 'bio14', 'bio15', 'bio16', 'bio17', 'bio18',
       'bio19', 'bdod', 'cec', 'cfvo', 'clay', 'nitrogen', 'phh2o', 'sand',
       'silt', 'soc', 'landCov',
       'Popdensity2010', 'Railways', 'Roads']].values
X2 = data_sans_doublons[['dayOfYear', 'bio1', 'bio2',
       'bio3', 'bio4', 'bio5', 'bio6', 'bio7', 'bio8', 'bio9', 'bio10',
       'bio11', 'bio12', 'bio13', 'bio14', 'bio15', 'bio16', 'bio17', 'bio18',
       'bio19', 'bdod', 'cec', 'cfvo', 'clay', 'nitrogen', 'phh2o', 'sand',
       'silt', 'soc', 'landCov',
       'Popdensity2010', 'Railways', 'Roads']].values

In [3]:
colonnes_avec_nan = blind_abiotique.columns[blind_abiotique.isna().any()]
colonnes_avec_nan

Index([], dtype='object')

In [8]:
data_sans_doublons = presences.groupby(['patchID'])['speciesId'].apply(list).reset_index()
data_sans_doublons = data_sans_doublons.merge(donnees_abiotiques, on='patchID', how='inner')
data_sans_doublons.dropna(inplace = True)
data_sans_doublons.reset_index(drop=True, inplace=True)

In [10]:
 # Création du modèle
nn = NearestNeighbors(n_neighbors=80)
nn.fit(X2)
# le modèle retourne l'indice des k-voisins ainsi que leurs distances du point
distances, indices = nn.kneighbors(X1)
# Récupération des espèces selon les indices des coordonnées des voisins
result = []
for i in range(len(X1)):
    nearest_species = {}
    for j in range(len(indices[i])):
        index = indices[i][j]
        species_list = data_sans_doublons.loc[index]['speciesId']
        for species in species_list:
            if species in nearest_species:
                nearest_species[species] += 1
            else:
                nearest_species[species] = 1
    selected_species = [species for species, count in nearest_species.items() if count > 80 * 0.2]
    result.append(list(selected_species))

# Traitement pour l'export du CSV au format acceptable par Kaggle
blind['nearest_species'] = result
blind = blind[['Id', 'nearest_species']]
blind = blind.rename(columns={'nearest_species': 'Predicted'})    
blind['Predicted'] = blind['Predicted'].apply(lambda x: str(x).replace('[', '').replace(']', '').replace(',', ''))
blind.to_csv("80voisinsabiotiqueproba02.csv", index=False, sep=',')
